<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# !pip3 install dash
# !pip3 install jupyter_dash
# !pip3 install dash_bootstrap_components

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import jupyter_dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objs as go
from datetime import date
from datetime import datetime

In [23]:
unis = pd.read_csv('World_University_Rankings.csv')

In [24]:
unis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373 entries, 0 to 1372
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   University              1373 non-null   object 
 1   Country                 1373 non-null   object 
 2   City                    1322 non-null   object 
 3   Region                  1373 non-null   object 
 4   Logo                    1373 non-null   object 
 5   Link                    1373 non-null   object 
 6   Type                    1373 non-null   object 
 7   Size                    1373 non-null   object 
 8   Faculty_count           1373 non-null   int64  
 9   International_Students  1373 non-null   int64  
 10  Students_count          1373 non-null   int64  
 11  Rank_2017               1373 non-null   int64  
 12  Rank_2018               1373 non-null   int64  
 13  Rank_2019               1373 non-null   int64  
 14  Rank_2020               1373 non-null   

In [25]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [26]:
x = np.logical_and(unis['Country']=='Russia', unis['Rank_2022']>0)
fig = px.scatter_mapbox(
        unis[x].iloc[:100,:], lon='Longitude', lat='Latitude', color='Rank_2022', hover_name='University', size_max=40, hover_data=['Country'], zoom=3)
fig.update_layout(mapbox_style="carto-positron")
fig.show()

In [27]:
app.layout = html.Div(children=[
    html.Div([
        html.H1('Ranking of Universities Around the World'),
        html.Div(dcc.Graph(id='map', figure={}))
    ]),

    html.Div([
        dcc.Slider(id='mapSlider',
                  min=2017,
                  max=2022,
                  value=2022,
                  step=None,
                  marks={i: str(i) for i in range(2017, 2023)}
                  ),
             
        dcc.Dropdown([str(region) for region in unis['Region'].unique()], placeholder='Select Region', id='map_region'),
        dcc.Dropdown(placeholder='Select Country', id='map_country'),
    ])
])

In [28]:
@app.callback(
    Output('map_country', 'options'),
    Output('map', 'figure'),
    Input('mapSlider', 'value'),
    Input('map_region', 'value'),
    Input('map_country', 'value')
)


def updateMap(year, region, country):
    df = unis[unis['Rank_' + str(year)] > 0]
    print
    zoom = 0.6

    if region:
        zoom = 1.8
        df = df[df['Region'] == region]

    if country:
        zoom = 3
        df = df[df['Country'] == country]

    fig = px.scatter_mapbox(df.iloc[:100,:],
                            lon='Longitude',
                            lat='Latitude',
                            color='Rank_'+str(year),
                            hover_name='University',
                            hover_data=['Country'],
                            zoom=zoom,
                            title = 'Ranking of Universities Around the World',
                            mapbox_style='carto-positron')

    countries = [{'label': str(i), 'value':str(i)} for i in df['Country'].unique()]
    
    return countries, fig

In [29]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>